In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# Load dataset
try:
    df = pd.read_csv("sales_data_sample.csv", encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv("sales_data_sample.csv", encoding='latin1')

tqdm.pandas(desc="Processing")

# Convert ORDERDATE and drop invalid dates
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'], errors='coerce')
df = df.dropna(subset=['ORDERDATE'])

# Feature Engineering from time
df['OrderWeekday'] = df['ORDERDATE'].dt.day_name()
df['OrderWeek'] = df['ORDERDATE'].dt.isocalendar().week

# Replace MONTH_ID with readable month names
month_map = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "October", 11: "November", 12: "December"
}
df['MONTH_ID'] = df['MONTH_ID'].map(month_map)
df.rename(columns={'MONTH_ID': 'OrderMonth'}, inplace=True)

# Add creative seasonal label
season_map = {
    'December': 'Festive-Winter', 'January': 'Winter', 'February': 'Chill-End',
    'March': 'Early-Spring', 'April': 'Spring', 'May': 'Bloom',
    'June': 'Hot-Start', 'July': 'Midsummer', 'August': 'Late-Summer',
    'September': 'Autumn-Start', 'October': 'Fall', 'November': 'Pre-Winter'
}
df['CreativeSeason'] = df['OrderMonth'].map(season_map)

# Handle missing values
df['TERRITORY'] = df['TERRITORY'].fillna("Unassigned")
df['STATE'] = df['STATE'].fillna("Unknown")
df['POSTALCODE'] = df['POSTALCODE'].fillna("Missing")

# Merge ADDRESSLINE2 into ADDRESSLINE1 (replace ADDRESSLINE1)
df['ADDRESSLINE2'] = df['ADDRESSLINE2'].fillna("")
df['ADDRESSLINE1'] = df['ADDRESSLINE1'] + ", " + df['ADDRESSLINE2']
df['ADDRESSLINE1'] = df['ADDRESSLINE1'].str.replace(", $", "", regex=True)
df.drop(['ADDRESSLINE2'], axis=1, inplace=True)

# Region grouping
def custom_region(country):
    regions = {
        'Americas': ['USA', 'Canada', 'Mexico'],
        'Europe': ['Germany', 'France', 'Italy', 'Spain', 'UK', 'Ireland'],
        'Asia': ['Japan', 'Singapore', 'Philippines', 'India', 'China'],
        'Oceania': ['Australia', 'New Zealand']
    }
    for region, countries in regions.items():
        if country in countries:
            return region
    return 'Other'
df['GeoRegion'] = df['COUNTRY'].apply(custom_region)

# Product category
df['ProductCategory'] = df['MSRP'].progress_apply(
    lambda x: 'Premium' if x >= 100 else ('Mid-Tier' if x >= 50 else 'Budget')
)

# Customer tiering
total_sales = df.groupby('CUSTOMERNAME')['SALES'].sum()
df['CustomerTier'] = df['CUSTOMERNAME'].progress_apply(
    lambda x: 'Gold' if total_sales[x] > 100000 else ('Silver' if total_sales[x] > 50000 else 'Bronze')
)

# Sales normalization
df['SalesZScore'] = (df['SALES'] - df['SALES'].mean()) / df['SALES'].std()
df['SalesTag'] = df['SalesZScore'].progress_apply(
    lambda x: 'Above Average' if x > 1 else ('Below Average' if x < -1 else 'Normal')
)

# Save final preprocessed file
filename = "preprocessed_graphication_final_v2.csv"
output_path = os.path.join(os.getcwd(), filename)
df.to_csv(output_path, index=False)

print(f"✅ Final preprocessed dataset saved at: {output_path}")


Processing: 100%|██████████| 1518/1518 [00:00<00:00, 605569.10it/s]

✅ Final preprocessed dataset saved at: /content/preprocessed_graphication_final_v2.csv
